# Reading and Writing Electronic Text
Spring 2024

Myrah Sarwar

***

## Assignment 5
### Predictive model text generation

I was interested in exploring books with unreliable narrators and wanted to try combining this with text generators. I mostly just wanted to see how noticeable the generated parts would be considering that the logic in the original is already not that credible to begin with. My guess was that it would probably not blend in naturally since the models being used are fairly small.

For my source texts, I chose a couple chapters from **_Crime and Punishment_ by Fyodor Dostoevsky** and **_Life of Pi_ by Yann Martel**. 
 ***

**Load text files**

In [7]:
dostoyevsky_p3ch6 = open("dostoyevsky_p3ch6.txt").read()
print(dostoyevsky_p3ch6)

“I don’t believe it, I can’t believe it!” repeated Razumihin, trying in perplexity to refute Raskolnikov’s arguments.

They were by now approaching Bakaleyev’s lodgings, where Pulcheria Alexandrovna and Dounia had been expecting them a long while. Razumihin kept stopping on the way in the heat of discussion, confused and excited by the very fact that they were for the first time speaking openly about it.

“Don’t believe it, then!” answered Raskolnikov, with a cold, careless smile. “You were noticing nothing as usual, but I was weighing every word.”

“You are suspicious. That is why you weighed their words… h’m… certainly, I agree, Porfiry’s tone was rather strange, and still more that wretch Zametov!... You are right, there was something about him—but why? Why?”

“He has changed his mind since last night.”

“Quite the contrary! If they had that brainless idea, they would do their utmost to hide it, and conceal their cards, so as to catch you afterwards…. But it was all impudent and car

In [155]:
martel_ch1 = open("martel_ch1.txt").read()
print(martel_ch1)

My suffering left me sad and gloomy.

Academic study and the steady, mindful practice of religion slowly wrought me back to life. I have kept up
with what some people would consider my strange religious practices. After one year of high school, I
attended the University of Toronto and took a double-major Bachelor's degree. My majors were religious
studies and zoology. My fourth-year thesis for religious studies concerned certain aspects of the cosmogony
theory of Isaac Luria, the great sixteenth-century Kabbalist from Safed. My zoology thesis was a functional
analysis of the thyroid gland of the three-toed sloth. I chose the sloth because its demeanour-calm, quiet and
introspective-did something to soothe my shattered self.

There are two-toed sloths and there are three-toed sloths, the case being determined by the forepaws of the
animals, since all sloths have three claws on their hind paws. I had the great luck one summer of studying the
three-toed sloth in situ in the equatorial jun

Shortened versions from each of the chapters above:

In [9]:
dostoyevsky_excerpt = "The old woman was a mistake perhaps, but she’s not the point! The old woman was merely a sickness . . . I was in a hurry to step over . . . it wasn’t a human being I killed, it was a principle! So I killed the principle, but I didn’t step over, I stayed on this side . . . All I managed to do was kill. And I didn’t even manage that, as it turns out . . ."
print(dostoyevsky_excerpt)

The old woman was a mistake perhaps, but she’s not the point! The old woman was merely a sickness . . . I was in a hurry to step over . . . it wasn’t a human being I killed, it was a principle! So I killed the principle, but I didn’t step over, I stayed on this side . . . All I managed to do was kill. And I didn’t even manage that, as it turns out . . .


In [317]:
martel_excerpt = "Richard Parker has stayed with me. I've never forgotten him. Dare I say I miss him? I do. I miss him. I still see him in my dreams. They are nightmares mostly, but nightmares tinged with love. Such is the strangeness of the human heart. I still cannot understand how he could abandon me so unceremoniously, without any sort of goodbye, without looking back even once. The pain is like an axe that chops my heart."
print(martel_excerpt)

Richard Parker has stayed with me. I've never forgotten him. Dare I say I miss him? I do. I miss him. I still see him in my dreams. They are nightmares mostly, but nightmares tinged with love. Such is the strangeness of the human heart. I still cannot understand how he could abandon me so unceremoniously, without any sort of goodbye, without looking back even once. The pain is like an axe that chops my heart.


## Markovify

Install Markovify + import library:

In [ ]:
import sys
!{sys.executable} -m pip install markovify

import markovify

Create generators using excerpts:

In [13]:
dostoyevsky_short_generator = markovify.Text(dostoyevsky_excerpt)
martel_short_generator = markovify.Text(martel_excerpt)

In [99]:
print(dostoyevsky_short_generator.make_sentence())

The old woman was merely a sickness . . I was in a hurry to step over . . I was in a hurry to step over . . . I was in a hurry to step over . .


I kind of like the repetitiveness in this one? It's still too similar to the source text though, so I'm trying this with a different state size. I think the ellipses are throwing it off.

In [105]:
dostoyevsky_short_generator_2 = markovify.Text(dostoyevsky_excerpt, state_size=1)
print(dostoyevsky_short_generator_2.make_sentence(test_output=False))

So I killed the point!


Still too similar for my liking. I'm going to try breaking it down further by character.

In [156]:
class SentencesByChar(markovify.Text):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Dostoevsky test using short and long texts:

In [159]:
dostoyevsky_gen_char = SentencesByChar(dostoyevsky_excerpt, state_size=4)
print(dostoyevsky_gen_char.make_sentence(test_output=False).replace("\n", " "))

So I kill.


In [165]:
dostoyevsky_full_gen_char = SentencesByChar(dostoyevsky_p3ch6, state_size=5)
print(dostoyevsky_full_gen_char.make_sentence(test_output=False).replace("\n", " "))

A fly flew by and the sofa and how annoying in from the door was soaked with shaking along while, and a wretched skinny old woman.


Martel test:

In [167]:
martel_gen_char = SentencesByChar(martel_excerpt, state_size=3)
print(martel_gen_char.make_sentence(test_output=False).replace("\n", " "))

Such is that chops my dreams.


In [182]:
martel_full_gen_char = SentencesByChar(martel_ch1, state_size=6)
print(martel_full_gen_char.make_sentence(test_output=False).replace("\n", " "))

Dare I say I miss the house lizards on their hind paws.


Seems that the full chapters are working better than my short versions, so I'm going to combine them.

In [249]:
dostoyevsky_full_gen = markovify.Text(dostoyevsky_p3ch6)
martel_full_gen = markovify.Text(martel_ch1)
combo = markovify.combine([dostoyevsky_full_gen, martel_full_gen], [0.5, 0.5])

In [256]:
print(combo.make_sentence())

My zoology thesis was a peculiar breathlessness in the long coat.


More output(s):

~~~
When you've suffered a great deal in life, but I don't believe in life, each additional pain is like an axe that chops at my side to remind me of God.
~~~

~~~
I killed the principle, but I love Canada.
~~~


Switching between words and characters to see if that makes a difference:

In [316]:
level = "word"

model_cls = markovify.Text if level == "word" else SentencesByChar
gen_a = model_cls(dostoyevsky_p3ch6, state_size=2)
gen_b = model_cls(martel_ch1, state_size=2)
gen_combo = markovify.combine([gen_a, gen_b], [0.5, 0.5])
for i in range(2):
    out = gen_combo.make_short_sentence(280, test_output=False)
    out = out.replace("\n", " ")
    print(out)

Not a sound came from the Department of Zoology.
That pain is both unbearable and trifling.


More output(s):

~~~
It’s too inartistic.
And perhaps he was he, what it was!

~~~

## Transformers

Import library and install:

In [ ]:
import sys
!conda install --prefix {sys.prefix} -y -c pytorch pytorch

In [ ]:
!{sys.executable} -m pip install transformers

In [157]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [158]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [159]:
vocab = tokenizer.get_vocab()

In [160]:
import torch

Other stuff I need to add to train the model. I'll use the Dostoyevsky text for this and then attach it to the Martel text after.

In [ ]:
import sys
!{sys.executable} -m pip install datasets

In [ ]:
pip install --upgrade aiohttp

In [54]:
import datasets

In [162]:
training_data = datasets.load_dataset('text', data_files="dostoyevsky_p3ch6.txt")

 Nevermind! This part keeps giving me an error and I can't figure out why. I think the updated version of the Hugging Face Transformers library has some differences in how it's used. I tried different codes but it kept giving me a new error after fixing the previous one.

In [163]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_training_data = training_data.map(
    lambda x: tokenizer(x['text']),
    remove_columns=["text"]
)

In [164]:
block_size = 64
# magic from https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_training_data = tokenized_training_data.map(
    group_texts,
    batched=True,
    batch_size=200
)

In [58]:
from transformers import Trainer, TrainingArguments

Error message told me to run this:

In [ ]:
pip install accelerate -U

In [165]:
from accelerate import Accelerator, DataLoaderConfiguration

dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
accelerator = Accelerator(dataloader_config=dataloader_config)

model = model.to(accelerator.device)

That seemed like it worked at first but... still no!

In [166]:
trainer = Trainer(model=model,
                  train_dataset=lm_training_data['train'],
                  args=TrainingArguments(
                      output_dir='distilgpt2-finetune-dostoyevsky',
                      num_train_epochs=1,
                      do_train=True,
                      do_eval=False
                  ),
                  tokenizer=tokenizer)

In [69]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=12, training_loss=3.7863972981770835, metrics={'train_runtime': 2.0589, 'train_samples_per_second': 44.197, 'train_steps_per_second': 5.828, 'total_flos': 1486125268992.0, 'train_loss': 3.7863972981770835, 'epoch': 1.0})

In [70]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}


In [75]:
generator("Two roads diverged in a yellow", max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: Placeholder storage has not been allocated on MPS device!

***
**It's not working out and I've given up for now, so back to using the model as is.**

In [167]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

Tried to make the red warnings go away when I run the code below this to print but it did not work :(

In [ ]:
!pip install shutup
import shutup;
shutup.please()

In [174]:
temp_count = 0.2

for i in range(4):
    print(generator("It wasn’t a human being I killed, it was a", bad_words_ids=tokenizer(["\n"]).input_ids, temperature=temp_count, max_length=40)[0]['generated_text'], 
          generator("\nI've never forgotten", temperature=temp_count, max_length=30)[0]['generated_text'], 
          generator("\nThe pain is like", temperature=temp_count, max_length=8)[0]['generated_text'])
    print('')
    temp_count += 1.3

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It wasn’t a human being I killed, it was a human being I killed, it was a human being I killed, it was a human being I killed, it was a human being 
I've never forgotten that the world of the game is a very different place than the world of the game. I've never forgotten that the world 
The pain is like a broken bone



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It wasn’t a human being I killed, it was a living being. As the story is, humanhood's human essence exists, and the human being cannot escape within as long as human 
I've never forgotten that it was during that period when I was at BYU, as I would never have imagined it had not existed in the early 
The pain is like getting punched one



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It wasn’t a human being I killed, it was a human being and no. Just like this is an age. It feels lonely in that I really think we have that sorts and 
I've never forgotten who those little bastard had was from what you went here over on Friday for lunch when he had us picket in the 
The pain is like the body tearing



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It wasn’t a human being I killed, it was a war that will destroy our civilizations that had made my culture disappear as many civilizations and civilization that took our peoples life into chaos until war 
I've never forgotten about any type of group identity – not so for as far out. Many, on occasion -- think on whether or even some 
The pain is like soapy,



More output(s):

~~~
It wasn’t a human being I killed, it was a human being I killed, it was a human being I killed, it was a human being I killed, it was a human being 
I've never forgotten that the world is a place where people are not allowed to be in a place where they can be in a place where they 
The pain is like a broken bone

It wasn’t a human being I killed, it was a human being. ․ It’ was a human being who created the world just as human beings are. It 
I've never forgotten to look back to my life. The day started by watching the TV for a minute or two, then I realized it was 
The pain is like a broken toe

It wasn’t a human being I killed, it was a family of children." said the court proceedings to the man during one hour with little else; a few, with the blessing, walked 
I've never forgotten just something else — how well it is possible every single act you may've made has one that brings on a life-saving 
The pain is like falling down the

It wasn’t a human being I killed, it was a boy named Saki and Heiki †I went along who lived to eat.’"I did it every two steps 
I've never forgotten an election in Ireland for whom I think there was something that we shouldn's want. People might think like they see politicians go 
The pain is like that, this
~~~

Keeping the beginning and end of the original excerpt and only replacing the middle with generated text:

In [185]:
print(generator("It wasn’t a human being I killed, it was a", bad_words_ids=tokenizer(["\n"]).input_ids, max_length=35)[0]['generated_text'])
print("All I managed to do was kill. And I didn’t even manage that, as it turns out . . .")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It wasn’t a human being I killed, it was a monster we'd never see.It didn’t happen. But it wasn’t a woman
All I managed to do was kill. And I didn’t even manage that, as it turns out . . .


In [196]:
print(generator("Richard Parker has stayed with me. I've never forgotten him.", bad_words_ids=tokenizer(["\n"]).input_ids, temperature=1.5, max_length=30)[0]['generated_text'])
print("I still see him in my dreams.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Richard Parker has stayed with me. I've never forgotten him. We both got on top of my heart and then at the end of my life,
I still see him in my dreams.
